### Run cell below to play Boxy!
##### Requires PyGame Module

In [1]:
import os
os.environ['PYGAME_HIDE_SUPPORT_PROMPT'] = "hide"

import pygame
if False:
    # These must be spcified for pyinstaller for some reason.
    import pygame._view
    import numpy.random.common
    import numpy.random.bounded_integers
    import numpy.random.entropy



pygame.init();

# Custom classes and functions
from Constants import display_size
from Keyboard import Game_Keyboard
from Play_Level import play_level

# Initialize game
gameDisplay = pygame.display.set_mode(display_size)
clock = pygame.time.Clock()

# initialize keyboard object to take in control data and feed it to the player object
keyboard = Game_Keyboard()

play_level(1,keyboard,gameDisplay,clock)

pygame.quit()


In [2]:
import numpy as np

np.linalg.norm([2,3])

3.605551275463989

To Do:

Add spring boxes (metal/wood, one class that adds a shape of an arrow, and that's it, rest will be handled outside)

Level saving/checkpoints

Game structure, saving, scores, etc.

Bad guys!

fruit and lives fly towards counters at the top when grabbed -- protectors fly from box?

Known bugs:
no handling of uncrouch for unbreakable objects (need to keep player crouched)
falling non-breakable boxes do not kill player
need a got-a-life sound
not a bug, but need to change lives for fruit in master list when they are got (avoid infinite life trick)


Credits:
“Sound effects obtained from zapsplat.com and freesound.org"
"Idea inspired by but unrelated to Crash Bandicoot"




### Run cell below to edit a level of Boxy!
##### Requires PyGame Module

In [ ]:
import pygame

pygame.init()

# Custom classes and functions
from Constants import display_size
from Level import Level
from Keyboard import Build_Keyboard
from Build_Level import build_level


# Initialize game
gameDisplay = pygame.display.set_mode([int(display_size[0]*1.3),display_size[1]])
clock = pygame.time.Clock()

# initialize keyboard object to take in control data and feed it to the player object
keyboard = Build_Keyboard()

build_level(0,keyboard,gameDisplay,clock)

pygame.quit()